<style>       
    hr{
        height: 4px;
        background-color: rgb(247,148,9);
        border: none;
    }
</style>
<div style="color=white;
           display:fill;
           border-radius:5px;
           background-color:rgb(34,41,49)">
<hr>
<div align="right"><i>BTE5034 - Digital Signal Processing &nbsp;</i></div>
<div align="right">EIT - BFH &nbsp;</div>

<div style="clear: both; font-size: 30pt; font-weight: bold;">
    Introduction to the discrete Fourier transform
</div>
<hr>
</div>

In [ ]:
# Import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import ipywidgets as wg

plt.rcParams["figure.figsize"] = (12,3)

In [ ]:
# These are helper functions for the interactive plots used later in the notebook. No need to study or understand
#  the code unless you are interested in advanced plotting and interactive widgets

UC_LIM = 1.4
UC_ARW = 0.1
UC_ARC = 0.8
UC_TXT = 1.2
UC_PSZ = 5

def wcolor(w):
    if np.isclose(w, 0) or np.isclose(w, np.pi):
        return 'C0'
    elif w > 0:
        return 'C2'
    else:
        return 'C3'
    
def arc(a=0, b=2*np.pi, r=1, pts=60, col='C0', arrow=False):
    theta = np.linspace(a, b, pts)
    plt.plot(r * np.cos(theta), r * np.sin(theta), color=col, linewidth=2)
    if arrow:
        d = np.sign(b) * UC_ARW
        x = r * np.array([np.cos(b-d), np.sin(b-d)])
        y = r * np.array([np.cos(b), np.sin(b)]) - x
        plt.arrow(*x, *y, shape='full', lw=1, length_includes_head=True, head_width=.08, color=col)

def ucsetup():
    plt.gcf().set_figheight(UC_PSZ)
    plt.gca().set_aspect('equal', 'box') 
    plt.gca().set(xlim=(-UC_LIM, UC_LIM), ylim=(-UC_LIM, UC_LIM))
    plt.axvline(0, linewidth=0.8, color='k')
    plt.axhline(0, linewidth=0.8, color='k')
    plt.axis('off')  
    arc()
    
def ucpoint(w, lb='', col='C2'):
    x, y = np.cos(w), np.sin(w)
    plt.plot([0, x], [0, y], color=col)
    plt.plot(x, y, 'o', color=col)
    plt.text(UC_TXT * x, UC_TXT * y, lb, ha='center', va='bottom' if np.round(y)==0 else 'center')          

def ctspin(w):
    w = np.fmod(w, 2 * np.pi)
    ucsetup()
    ucpoint(w, f'$e^{{j\\theta}}$', wcolor(w))
    arc(0, w, r=UC_ARC, col='C1', arrow=True)
    return w

def dtspin(N, k, n=None):
    ucsetup()
    w = 2 * np.pi / N * k
    for m in range(n+1 if n is not None else N):
        ucpoint(w * m, f'$w_{{{k}}}[{m}]$', wcolor(np.pi - w))
    ab = [0, w] if n is None else [w * (n-1), w * n]
    arc(*ab, r=UC_ARC, col='C1', arrow=True)

def ccspin(N, k, n=1):
    ucsetup()
    w = 2 * np.pi / N * k * n
    ucpoint(w, f'$w_{{{k}}}[{n}]$')
    ucpoint(-w, f'$w_{{{N-k}}}[{n}]$', 'C3')
    plt.plot([np.cos(w), np.cos(w)], [np.sin(w), -np.sin(w)], 'C6')
    plt.plot([np.cos(w), np.cos(w)], [np.sin(w), -np.sin(w)], 'oC6')
    plt.plot([0, np.cos(w)], [0, 0], 'C9')
    plt.plot(np.cos(w), 0, 'oC9')
  
def show_cex(N:int, k:int):
    w = 2 * np.pi / N * k
    cex = np.round(np.exp(1j * w * np.arange(N)), 3)
    plt.subplot(1,2,1)
    ucsetup()
    arc(0, w, r=UC_ARC, col='C1', arrow=True)    
    pts = {np.round(np.fmod(w * n, 2 * np.pi), 2): n for n in range(N-1,-1,-1)}
    for p, n in pts.items():
        ucpoint(n * w, f'$w_{{{k}}}[{n}]$', 'C2' if n < 2 else 'lightgray')
    plt.subplot(2,2,2)
    plt.stem(cex.real)
    plt.title(f'$\\mathrm{{Re}}\\{{\\mathbf{{w}}_{{{k}}}\\}} = \\mathbf{{c}}_{{{k}}}$')
    plt.subplot(2,2,4)
    plt.stem(cex.imag)
    plt.title(f'$\\mathrm{{Im}}\\{{\\mathbf{{w}}_{{{k}}}\\}} = \\mathbf{{s}}_{{{k}}}$')
    plt.tight_layout()

def show_freqs(N:int):
    ucsetup()    
    for k in range(N):
        w = 2 * np.pi / N * k
        if k < N / 2:
            ucpoint(w, f'$\\omega_{{{k}}}$', wcolor(w))
        elif k == N / 2:
            ucpoint(np.pi, f'$\\omega_{{{k}}} = \\pi$', wcolor(0))
        else:
            ucpoint(w, f'$\\omega_{{{k}}} = -\\omega_{{{N-k}}}$', wcolor(np.pi - w))
    plt.title(f'$N = {N}$')

def display_sbs(interactive_widget):
    c = interactive_widget.children
    controls = wg.HBox(c[:-1], layout=wg.Layout(align_items='center'))
    return display(wg.HBox([controls, c[-1]]))

# Bulding signals out of oscillations

The fundamental intuition of Fourier analysis is that a signal, no matter how complicated its "shape", can always be represented as a weighted sum of sinusoidal oscillations; a Fourier transform is the method to compute the value of the necessary weights. 

This is a very old idea and something that in the past was often implemented with mechanical or electrical devices, as for example in the famous tide prediction machine designed by Lord Kelvin in 1872:

<img width="449" src="img/tidemachine.jpg">

Using DSP blocks, this machine would look like this:

<img width="449" src="img/IDFT.png">

## The complex exponential

The expression $e^{j\theta}$ represents a point on the unit circle in the complex plane placed at an angle of $\theta$ radians measured _counterclockwise_ from the horizontal axis. The Cartesian coordinates of this point on the plane are its real and imaginary parts, that is, 
$$
    e^{j\theta} = \cos\theta + j\sin\theta.
$$

Note that:
 * the position of the point is a $2\pi$-periodic function of the angle: $e^{j(\theta + 2m\pi)} = e^{j\theta}$ for any $m \in \mathbb{Z}$
 * if the angle is negative, the position of the point is measured _clockwise_ from the real axis
 * complex conjugation flips the sign of the angle: $(e^{j\theta})^* = e^{-j\theta}$

Because of the $2\pi$-periodicity, it is customary to _wrap_ the phase of a complex exponential over the interval $[-\pi, \pi]$; indeed, any point on the upper half of the unit circle can be reached via a **counterclockwise** rotation with $0 \le \theta \le \pi$ radians while points on the lower half can be reached via a **clockwise** rotation with $-\pi \le \theta \le 0$

In [ ]:
ws = wg.FloatSlider(value=0, min=-15, max=15, description='$\\theta$')
display_sbs(wg.interactive(ctspin, w=ws))

## Complex-valued harmonic oscillations

The family $W_N = \{\mathbf{w}_k\}_{0 \le k < N}$ is a set of $N$ discrete-time, complex-valued, harmonic oscillations where
$$
    w_k[n] = e^{j\omega_k n} = \cos(\omega_k n) + j \sin(\omega_k n), \qquad \omega_k = \frac{2\pi}{N}k
$$
The $N$ samples in $\mathbf{w}_k$ describe the position of a point on the complex plane that rotates _counterclockwise_ around the unit circle in angular increments of $\omega_k$ radians; the Cartesian coordinates of $w_k[n]$ are thus the cosine and the sine of the angle $\omega_k n$:

With NumPy, complex-valued computations are straightforward and we can generate the elements in $W_N$ as:

In [ ]:
def cex_N(N:int, k:int, n:np.ndarray=None) -> np.ndarray:
    n = np.arange(N) if n is None else n
    return np.exp(2j * np.pi / N * k * n)

The following interactive plot displays the samples in $\mathbf{w}_k$ as points on the complex plane and as plots of their real and imaginary parts (if multiple points fall on the same position, only the first label is shown):

In [ ]:
N = 16
display_sbs(wg.interactive(show_cex, N=wg.fixed(N), k=wg.IntSlider(1, 1, N-1)))

## Building signals with complex exponentials

### A discrete-time square wave

A balanced, discrete-time square wave with period $N$ samples is defined over $[0, N-1]$ as
$$
    q[n] = \begin{cases} 0 & n =0, N/2 \\ 1 & 0 < n < N/2 \\ -1 & N/2 < n < N \end{cases}
$$
and prolonged by periodicity over $\mathbb{Z}$. We assume $N$ even, so that $N/2$ is an integer; the square wave is antisymmetric around its midpoint $N/2$, hence the adjective of "balanced".

In Python, we can easily implement a generator for this signal using the modulo operation:

In [ ]:
def sqw(N: int, n: np.ndarray=None) -> np.ndarray:
    # if no index range then return the values over [0, N-1]    
    n = np.arange(N) if n is None else n
    q = np.ones_like(n).astype(float)
    q[np.mod(n, N/2) == 0] = 0
    q[np.mod(n, N) > N/2] *= -1
    return q

Here is the first period of the square wave

In [ ]:
N = 64
plt.stem(sqw(N));

but we can also plot as many periods as we like:

In [ ]:
P = 12
n = np.arange(-P * N, P * N)
plt.plot(n, sqw(N, n));


The square wave can be obtained as 
$$
    \mathbf{q} = \frac{1}{N} \sum_{k=0}^{N-1} Q[k] \mathbf{w}_k 
$$
with
$$
    Q[k] = \begin{cases} 0 & k \text{~even} \\ \displaystyle \frac{-2j}{\tan(\pi k /N)}  & k \text{~odd} \end{cases}
$$

In [ ]:
def sqw_idft(N: int, n: np.ndarray=None) -> np.ndarray:
    n = np.arange(N) if n is None else np.mod(n, N)
    q = np.fft.ifft([-2j / np.tan(np.pi / N * k) if k % 2 == 1 else 0 for k in range(N)]).real
    return q[n]

In [ ]:
plt.stem(sqw_idft(100));

In [ ]:
M = 1002
np.allclose(sqw_idft(M), sqw(M))

### A discrete-time triangular wave

A discrete-time triangular signal with period $N$ samples is defined over $[0, N-1]$ as
$$
    t[n] = \frac{4}{N}\left|n - \frac{N}{2}\right| - 1
$$
and prolonged by periodicity over $\mathbb{Z}$.

Complete the code below to implement a generator for the periodic triangular signal.

In [ ]:
def tri(N: int, n: np.ndarray=None) -> np.ndarray:
    n = np.arange(N) if n is None else n
    t = (4 / N) * np.abs(np.mod(n, N) - N / 2) - 1 
    return t

In [ ]:
N = 64
plt.stem(tri(N));

In [ ]:
P = 10
n = np.arange(-P * N, P * N)
plt.plot(n, tri(N, n));

### Exercise: triangular wave from complex oscillations

The square wave can be obtained as 
$$
    \mathbf{q} = \frac{1}{N} \sum_{k=0}^{N-1} T[k] \mathbf{w}_k 
$$
with
$$
    T[k] = \begin{cases} 0 & k \text{~even} \\ \displaystyle \frac{4}{N}\,\left(1 + \frac{1}{\tan^2(\pi k/N)}\right) & k \text{~odd} \end{cases}
$$
Complete the code below to generate a triangular wave using complex oscillations

In [ ]:
def tri_idft(N: int, n: np.ndarray=None) -> np.ndarray:
    n = np.arange(N) if n is None else np.mod(n, N)
    ... # your code here
    return t[n]

Using the same reasoning as before, but considering that the triangular wave is symmetric, we can write


In [ ]:
M = 14
np.allclose(tri_idft(M), tri(M))

In [ ]:
plt.stem(sqw(100));

In [ ]:
def dft_coef_sqw(N):
    return np.array([-2j / np.tan(np.pi / N * k) if k % 2 == 1 else 0 for k in range(N)])

In [ ]:
plt.stem(dft_coef_sqw(100).imag);

In [ ]:
plt.stem(np.fft.fft(sqw(100)).imag);

In [ ]:
np.allclose(dft_coef_sqw(100).imag, np.fft.fft(sqw(100)).imag)

In [ ]:
plt.stem(np.fft.ifft(np.fft.fft(sqw(100))))

In [ ]:
N = 105
w = 2 * np.pi * 0.1
x = np.cos(w * np.arange(N)) 

In [ ]:
plt.stem(x)

 1. compute the DFT of the cosine for N = 100
 2. plot (stem) the absolute values of the DFT coefficients
 3. try to understand why they look like that
 4. repeat for N = 105
    

In [ ]:
plt.stem(np.abs(np.fft.fft(x)));

In [ ]:
N = np.linspace(10, 1000000, 100)
#plt.plot(N**2)
plt.plot(N * np.log2(N) / (N**2))